In [ ]:
import sys
import pandas as pd
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Importation de vos fonctions "maison"
from src.models import run_har_comparison
from src.evaluation import run_walk_forward

print(f"Project Root: {project_root}")

In [ ]:

data_dir = project_root / "data" / "processed" / "model_datasets"


targets = [
    ("WTI", "wti_dataset.csv"),
    ("GAS", "gas_dataset.csv"),
    ("GOLD", "gold_dataset.csv")
]

for name, file_name in targets:
    path = data_dir / file_name
    status = "Found" if path.exists() else "Missing (Run main.py step 4 first)"
    print(f"{name}: {status}")

In [ ]:
print("=== STEP 1: IN-SAMPLE DIAGNOSTIC (HAR vs HAR-X) ===\n")

for name, file_name in targets:
    file_path = data_dir / file_name
    
    if file_path.exists():

        # Elle imprime directement les tableaux de résultats
        run_har_comparison(file_path, name)
    else:
        print(f"Skipping {name} (File not found)")

In [ ]:
print("=== STEP 2: OUT-OF-SAMPLE FORECASTING (WALK-FORWARD) ===\n")
print("Note: This step usually takes a few minutes per commodity.\n")

# Paramètres de la fenêtre glissante (identiques à votre main.py)
WINDOW_SIZE = 750
STEP_SIZE = 5         # Prévision tous les 5 jours pour aller plus vite
START_DATE = "2015-01-01"
END_DATE = "2024-12-31"

for name, file_name in targets:
    file_path = data_dir / file_name
    
    if file_path.exists():
        print(f"\nRunning Forecasts for {name}...")
        
        results_df = run_walk_forward(
            file_path=file_path,
            commodity_name=name,
            window_size=WINDOW_SIZE,
            step_size=STEP_SIZE,
            start_date=START_DATE,
            end_date=END_DATE)
            
        print(f"Done. Rows processed: {len(results_df)}")
        
    else:
        print(f"Skipping {name} (File not found)")